<a href="https://colab.research.google.com/github/mjayashri/llm_from_scratch/blob/main/building_llm_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -q ipdb
!pip install -q tqdm
!pip install -q sentencepiece
!pip install -q wandb

In [10]:
import os, sys
import ipdb
from tqdm import tqdm
from datetime import datetime
import platform, shutil
import requests, zipfile, io

#pytorch
import torch
import torch.nn as nn
from torch.nn import functional as F

#tokenizer
import sentencepiece as spm

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

torch.cuda.empty_cache()

In [11]:
files_url = "https://ideami.com/llm_train"
print("Downloading files...")

response = requests.get(files_url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))
zip_file.extractall(".")
print("Done!")

Done!


In [12]:
# Architecture parameters
batch_size = 128
context = 512
embed_size = 384

n_layers = 7
n_heads = 7
BIAS = True

# Hyperparameters
lr = 3e-4
dropout = 0.05
weight_decay = 0.01
grad_clip = 1.0

# Training parameters
train_iters = 100000
eval_interval = 50
eval_iters = 10
compile = False
checkpoint_dir = "models"
checkpoint_fn = "latest.pt"
checkpoint_load_fn = "latest.pt"
dtype = torch.bfloat16

#mode

inference = False

#Device

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device : You will be running {device}")


device : You will be running cpu


In [14]:
# Logging

wandb_log = True
wandb_project = "llm1"

wandb_run_name = "llm1-"+datetime.now().strftime("%Y%m%d-%H%M%S")
print(f"wandb_run_name : {wandb_run_name}")

if wandb_log:
    import wandb
    # wandb.login()
    wandb.init(project=wandb_project, name=wandb_run_name)


wandb_run_name : llm1-20241117-184842


In [17]:
with open("wiki.txt", 'r', encoding='utf-8') as f:
    text = f.read()

print(text[5000:5300])

tely before leap years, October of the following year. In years immediately before common years, August finishes on the same day of the week as May of the following year, and in years immediately before leap years, February and October of the following year.




Art

Art is a creative activity. It p


In [18]:
from ast import mod
# tokenizer

sp = spm.SentencePieceProcessor(model_file='wiki_tokenizer.model')
vocab_size = sp.get_piece_size()
print('vocab size:', vocab_size)

vocab size: 4096


In [19]:
encode = lambda s: sp.Encode(s)
decode = lambda l: sp.Decode(l)

print(encode("hello world"))
print(decode(encode("hello world")))

[385, 840, 4037, 969]
hello world


In [20]:
if os.path.exists("encoded_data.pt"):
  print("Loading encoded data...")
  data = torch.load("encoded_data.pt")
  print("Done!")
else:
  print("Encoding data...")
  data = torch.tensor(encode(text), dtype=torch.long)
  torch.save(data, "encoded_data.pt")
  print("Done!")

Loading encoded data...


<ipython-input-20-d7a22881af3a>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("encoded_data.pt")


Done!
